In [1]:
import ingestor
import extractor2

import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2
1.14.0


In [2]:
import data
import utils

# Research notebook - Jeff
* python 3.6.5
* tensorflow 1.14
* conda_tensorflow_p36


## Making a vanilla NN after the extractor

In [85]:
# ingestor
input_fn, params = ingestor.get_input_fn()

INFO:tensorflow:PASSED IN parser is None


In [76]:
features, labels = input_fn()[0], input_fn()[1]

In [78]:
# extractor
features, labels = extractor2.Extractor(params).extractor(features, labels)

In [99]:
def manual_iterator():
    input_fn, params = ingestor.get_input_fn()
    features, labels = input_fn(), input_fn()
    features, labels = extractor2.Extractor(params).extractor(features, labels)
    
    goals_X = features['goal_ids']
    tacid_y = labels['tac_id']
    
    return goals_X, tacid_y

# Vanilla NN

In [45]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

In [46]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
(x_train, y_train), (x_test, y_test)

In [82]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

---

# Running extraction pipeline

In [4]:
! pip install progressbar

  Running setup.py bdist_wheel for progressbar ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
import importlib
import extraction_pipeline
import os
import pandas as pd
import numpy as np

import importlib
importlib.reload(extraction_pipeline)

<module 'extraction_pipeline' from '/home/ec2-user/SageMaker/deepmath/deepmath/deephol/train/extraction_pipeline.py'>

In [24]:
# run
extraction_pipeline.run_extraction_pipeline(data_split='train')

100% |########################################################################|


Success: extracting data into arrays for goals, thms and tactics


KeyboardInterrupt: 

In [ ]:
'done'

## Read saved transformed data

In [6]:
processed_path = 's3://sagemaker-cs281/deephol-data-processed/proofs/human/train'



In [7]:
goals_path = os.path.join(processed_path, 'goal_ids.csv')

In [8]:
# read files
goal_ids = pd.read_csv(goals_path)

In [10]:
len(goal_ids)

9999

# upload them

In [20]:
import pandas as pd
import numpy as np
import sys
import boto3
import numpy as np

BUCKET_NAME = 'sagemaker-cs281'

In [21]:
def upload_np_to_s3(array, object_name):    
    # save localy
    local_filename = '/tmp/temp.csv'
    np.savetxt(local_filename, array, delimiter=',')
    
    # s3 upload
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(local_filename, BUCKET_NAME, object_name)

In [22]:
paths = {
    'train':'deephol-data-processed/proofs/human/train',
    'valid':'',
    'test':''
}

In [27]:
# upload
upload_np_to_s3(labels_array, os.path.join(paths['train'], 'label3.csv'))

In [32]:
len(hypotheses_array[3])

7

In [33]:
print([sys.getsizeof(x) for x in [goals_array, hypotheses_array, labels_array]])

[400112, 896, 896]


In [50]:
a = pd.DataFrame(hypotheses_array)

In [ ]:
features_raw = np.array(features['goal_ids'])

In [97]:
import pickle

## tfrecord

In [14]:
def tf_serialize_example(*args):
    tf_string = tf.py_function(
        serialize_example,
        (*args),  # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar

In [15]:
labels_datset = tf.data.Dataset.from_tensor_slices(labels['tac_id'])

NameError: name 'labels' is not defined

In [106]:
features_dataset = tf.data.Dataset.from_tensor_slices(
    labels['tac_id'])


In [ ]:
# serialized_features_dataset = features_dataset.map(tf_serialize_example)
# serialized_features_dataset

In [ ]:
# filename = './test.tfrecord'
# writer = tf.data.experimental.TFRecordWriter(filename)
# writer.write(labels_datset)